Most of the benchmark definition and data loading happens over the next few cells.
If you are just interested in the results, please continue until the "Visualizations" header.

In [1]:
import openml
import pandas as pd

In [2]:
regression_datasets = [
    41021,
    574,
    550,
    546,
    541,
    531,
    507,
    505,
    287,
    216,
    183,
    42208,
    41540,
    42225,
    42571,
    4549,
    42572,
    42570,
    42708,
    42705,
    42688,
]

old_classification = [
    41142, 41163, 41161, 41167, 41147, 41165, 41150, 41164, 41143, 41146, 1169, 41138, 1111, 1596, 1486, 40996, 41027, 40685, 23512, 40981, 41166, 41168, 40668, 23517, 1489, 1468, 1461, 40984, 40975, 41159, 1590, 1464, 41169, 4135, 5, 12, 2, 3, 54, 31, 1067,
]

new_classification = [
    41164,
41146,
41145,
41144,
41143,
41142,
181,
1457,
40498,
41156,
41157,
41158,
41163,
40900,
    41162,
]

classification = old_classification + new_classification

dataset_suites = {
    "Regression": openml.datasets.list_datasets(regression_datasets, output_format="dataframe"),
    "Classification": openml.datasets.list_datasets(classification, output_format="dataframe"),
    "Classification (old)": openml.datasets.list_datasets(old_classification, output_format="dataframe"),
    "Classification (new)": openml.datasets.list_datasets(new_classification, output_format="dataframe"),
}

In [3]:
datasets = openml.datasets.list_datasets(regression_datasets, output_format="dataframe")

In [4]:
datasets.columns

Index(['did', 'name', 'version', 'uploader', 'status', 'format',
       'MajorityClassSize', 'MaxNominalAttDistinctValues', 'MinorityClassSize',
       'NumberOfClasses', 'NumberOfFeatures', 'NumberOfInstances',
       'NumberOfInstancesWithMissingValues', 'NumberOfMissingValues',
       'NumberOfNumericFeatures', 'NumberOfSymbolicFeatures'],
      dtype='object')

In [6]:
%matplotlib inline
import seaborn as sns
import matplotlib.pyplot as plt
import mpld3
import numpy as np
mpld3.enable_notebook()


from warnings import simplefilter
from matplotlib.cbook.deprecation import MatplotlibDeprecationWarning
# ignore all future warnings
simplefilter(action='ignore', category=MatplotlibDeprecationWarning)

In [10]:
import ipywidgets as widgets
from ipywidgets import Layout

# Visualizations

In [11]:
x_var = widgets.Select(
    options=list(datasets.columns),
    value="NumberOfInstances",
    rows=1,
    description='X:',
)

y_var = widgets.Select(
    options=list(datasets.columns),
    value="NumberOfFeatures",
    rows=1,
    description='Y:',
)

log_x = widgets.Checkbox(
    value=False,
    description='log x',
    disabled=False,
    indent=False,
    layout=Layout(width="75px"),
)

log_y = widgets.Checkbox(
    value=False,
    description='log y',
    disabled=False,
    indent=False,
    layout=Layout(width="75px"),
)

exclude = widgets.SelectMultiple(
    options=["None"] + list(datasets["name"]),
    value=["None"],
    description='Exclude',
    disabled=False
)

suite = widgets.Select(
    options=["Regression", "Classification", "Classification (old)", "Classification (new)"],
    value="Regression",
    rows=1,
    description='Set:',
)

def update_suite_selection(change):
    global datasets       
    datasets = dataset_suites[change.new]
    exclude.options = ["None"] + list(datasets["name"])
    exclude.value = ["None"]
    
suite.observe(update_suite_selection, names='value')

def plot(x, y, logx, logy, exclude):
    fig, ax = plt.subplots()
    filtered = datasets[~datasets["name"].isin(exclude)]
    # for some reason ax.set(xscale="log") does not work and shows points all over.
    xval = np.log10(filtered[x]) if logx else filtered[x]
    yval = np.log10(filtered[y]) if logy else filtered[y]
    ax = sns.scatterplot(x=xval, y=yval, data=filtered, ax=ax)
    labels=[f"{name} ({xi}, {yi})" for name, xi, yi in zip(filtered["name"], filtered[x], filtered[y])
            if not (np.isnan(xi) or np.isnan(yi))]
    tooltip = mpld3.plugins.PointLabelTooltip(ax.get_children()[0], labels=labels)
    mpld3.plugins.connect(fig, tooltip)


out = widgets.interactive_output(plot, {'x': x_var, 'y': y_var, 'logx': log_x, 'logy': log_y, 'exclude': exclude})
widgets.HBox([widgets.VBox([suite, exclude, x_var, y_var, widgets.HBox([log_x, log_y])]), out])

In [100]:
dataset_suites["Regression"]

,did,name,version,uploader,status,format,MajorityClassSize,MaxNominalAttDistinctValues,MinorityClassSize,NumberOfClasses,NumberOfFeatures,NumberOfInstances,NumberOfInstancesWithMissingValues,NumberOfMissingValues,NumberOfNumericFeatures,NumberOfSymbolicFeatures
183,183,abalone,1,1,active,ARFF,689.0,28.0,1.0,28.0,9.0,4177.0,0.0,0.0,7.0,2.0
216,216,elevators,1,1,active,ARFF,NaN,NaN,NaN,0.0,19.0,16599.0,0.0,0.0,19.0,0.0
287,287,wine_quality,1,94,active,ARFF,NaN,NaN,NaN,0.0,12.0,6497.0,0.0,0.0,12.0,0.0
505,505,tecator,1,2,active,ARFF,NaN,NaN,NaN,0.0,125.0,240.0,0.0,0.0,125.0,0.0
507,507,space_ga,1,2,active,ARFF,NaN,NaN,NaN,0.0,7.0,3107.0,0.0,0.0,7.0,0.0
531,531,boston,1,2,active,ARFF,NaN,9.0,NaN,0.0,14.0,506.0,0.0,0.0,12.0,2.0
541,541,socmob,1,2,active,ARFF,NaN,17.0,NaN,0.0,6.0,1156.0,0.0,0.0,2.0,4.0
546,546,sensory,1,2,active,ARFF,NaN,6.0,NaN,0.0,12.0,576.0,0.0,0.0,1.0,11.0
550,550,quake,2,2,active,ARFF,NaN,NaN,NaN,0.0,4.0,2178.0,0.0,0.0,4.0,0.0
574,574,house_16H,1,2,active,ARFF,NaN,NaN,NaN,0.0,17.0,22784.0,0.0,0.0,17.0,0.0
